## Vep results VCF details

![Vep results design](design.png)

To run this notebook yourself you will need juypter hub `pip install jupyterlab`, vcfpy `pip install vcfpy`, and a vep results vcf file. 

This notebook was generated using `/nfs/production/*/*/web/newsite-vep/vep-output-phase1-options-plus-con.vcf`

[VEP design XD](https://xd.adobe.com/view/2e253ea8-6bd8-4efa-ad39-dc65f9eb01bd-7b76/screen/802e46d2-1597-4821-ad78-20b392e5d925/?fullscreen)

In [1]:
import vcfpy
import json

def pretty_obj(d):
    print(json.dumps(d, sort_keys=True, indent=4))

[vcfpy](https://vcfpy.readthedocs.io/en/stable/index.html) is the python module used by variation to parse VCF files.

The main object used is  [record](https://vcfpy.readthedocs.io/en/stable/api_record.html)

For varient we should use `record.ID` which is an array of zero or more Ids. If there are no ids we should show `.`. In the design we show RS ids, but it has been confirmed that we should show whatever the use provided in their input VCF 


In [2]:
def open_vcf(): #You can only iterate once through a vcf
    vcf_file = "vep-output-phase1-options-plus-con.vcf"
    return vcfpy.Reader.from_path(vcf_file)

In [19]:
records = open_vcf()
counter = 0
print("ID\tREF\tLocation\tAlt")

for rec in records:
    id = "."
    if len(rec.ID) > 0:
        id = ", ".join(rec.ID) #VCF supports multiple semicolon delimited ids
        
    print(f"{id}\t{rec.REF}\t{rec.CHROM}{rec.begin}\t{', '.join([alt.value for alt in rec.ALT])}\t{rec.end}")

    counter += 1
    if counter >= 10:
        break


ID	REF	Location	Alt
.	C	chr1982663	T	None
.	T	chr1982828	A	None
.	C	chr1982866	T	None
.	T	chr1983371	C	None
.	A	chr1983452	G	None
.	C	chr1984187	T	None
.	T	chr1984299	C	None
.	G	chr1984652	A	None
.	C	chr1985276	A	None
.	A	chr19243500	G	None


- Most of the data we want is found in the INFO column of the VCF in a value called CSQ.
- There will be a CSQ entry for every ALT allele in a given record.
- The CSQ is `|` pipe delimited with null values showing as empty `||`.
- values can contain multiple results. They are delimited by an ampersand &



In [4]:
reader = open_vcf()
print(reader.header.get_info_field_info("CSQ").description)


Consequence annotations from Ensembl VEP. Format: Allele|Consequence|IMPACT|SYMBOL|Gene|Feature_type|Feature|BIOTYPE|EXON|INTRON|HGVSc|HGVSp|cDNA_position|CDS_position|Protein_position|Amino_acids|Codons|Existing_variation|REF_ALLELE|UPLOADED_ALLELE|DISTANCE|STRAND|FLAGS|SYMBOL_SOURCE|HGNC_ID|CANONICAL|SIFT|PolyPhen|AF|CLIN_SIG|SOMATIC|PHENO|MOTIF_NAME|MOTIF_POS|HIGH_INF_POS|MOTIF_SCORE_CHANGE|TRANSCRIPTION_FACTORS


In [5]:
reader = open_vcf()
csq_header_str = reader.header.get_info_field_info("CSQ").description.split(":")[-1].strip()
csq_headers = csq_header_str.split("|")
pretty_obj(csq_headers)

def csq_obj(headers,values):  
    return {headers[x]:values[x] for x in range(0,len(headers))}

[
    "Allele",
    "Consequence",
    "IMPACT",
    "SYMBOL",
    "Gene",
    "Feature_type",
    "Feature",
    "BIOTYPE",
    "EXON",
    "INTRON",
    "HGVSc",
    "HGVSp",
    "cDNA_position",
    "CDS_position",
    "Protein_position",
    "Amino_acids",
    "Codons",
    "Existing_variation",
    "REF_ALLELE",
    "UPLOADED_ALLELE",
    "DISTANCE",
    "STRAND",
    "FLAGS",
    "SYMBOL_SOURCE",
    "HGNC_ID",
    "CANONICAL",
    "SIFT",
    "PolyPhen",
    "AF",
    "CLIN_SIG",
    "SOMATIC",
    "PHENO",
    "MOTIF_NAME",
    "MOTIF_POS",
    "HIGH_INF_POS",
    "MOTIF_SCORE_CHANGE",
    "TRANSCRIPTION_FACTORS"
]


The values found in the CSQ will change depending on the options selected during VEP. Some values will always be present. We should check this header to ensure we are looking in the correct index before pulling out the values

Predicted molecular consequence can be found in `Consequence`. This is a multi value field so you will need to split by '&' if present

In [6]:
print("Predicted molecular consequence & alt allele frequency\n")

reader = open_vcf()

found = False
for rec in reader:
    for str_csq in rec.INFO["CSQ"]:
        csq_dict = csq_obj(csq_headers,str_csq.split("|"))
        if csq_dict['AF']:
            print(f"{rec.REF}\t{rec.CHROM}{rec.begin}\t{', '.join([alt.value for alt in rec.ALT])}")
            print(f"{', '.join(csq_dict['Consequence'].split('&'))}\t{csq_dict['AF']}")      
            found = True
            #break
    if found:
        break

Predicted molecular consequence & alt allele frequency

A	chr19243500	G
intron_variant, non_coding_transcript_variant	0.3730
A	chr19243500	G
upstream_gene_variant	0.3730
A	chr19243500	G
upstream_gene_variant	0.3730
A	chr19243500	G
intron_variant, non_coding_transcript_variant	0.3730
A	chr19243500	G
intron_variant, non_coding_transcript_variant	0.3730
A	chr19243500	G
regulatory_region_variant	0.3730


In [7]:
reader = open_vcf()

for rec in reader:
    csq_dict = csq_obj(csq_headers,rec.INFO["CSQ"][0].split("|"))
    pretty_obj(csq_dict)
    break
        

{
    "AF": "",
    "Allele": "T",
    "Amino_acids": "",
    "BIOTYPE": "lncRNA",
    "CANONICAL": "YES",
    "CDS_position": "",
    "CLIN_SIG": "",
    "Codons": "",
    "Consequence": "upstream_gene_variant",
    "DISTANCE": "4978",
    "EXON": "",
    "Existing_variation": "rs868831437",
    "FLAGS": "",
    "Feature": "ENST00000631376.1",
    "Feature_type": "Transcript",
    "Gene": "ENSG00000282591",
    "HGNC_ID": "HGNC:33581",
    "HGVSc": "",
    "HGVSp": "",
    "HIGH_INF_POS": "",
    "IMPACT": "MODIFIER",
    "INTRON": "",
    "MOTIF_NAME": "",
    "MOTIF_POS": "",
    "MOTIF_SCORE_CHANGE": "",
    "PHENO": "",
    "PolyPhen": "",
    "Protein_position": "",
    "REF_ALLELE": "C",
    "SIFT": "",
    "SOMATIC": "",
    "STRAND": "-1",
    "SYMBOL": "FAM138F",
    "SYMBOL_SOURCE": "HGNC",
    "TRANSCRIPTION_FACTORS": "",
    "UPLOADED_ALLELE": "C/T",
    "cDNA_position": ""
}


Gene & regulation features and transcripts

`Feature` and `Feature_type` hold details for both features and transcripts. It was agreed in the web variation catch up that anything with a `Feature_type` of `Transcript` will go in the transcript column for the design. Everything else should do in the features column

For transcripts we also show the `BIOTYPE`

In [8]:
print("Transcripts")

reader = open_vcf()

found = False
for rec in reader:
    for str_csq in rec.INFO["CSQ"]:
        csq_dict = csq_obj(csq_headers,str_csq.split("|"))
        if csq_dict['Feature'] and csq_dict['Feature_type'] == "Transcript":
            print(f"{rec.REF}\t{rec.CHROM}{rec.begin}\t{', '.join([alt.value for alt in rec.ALT])}")
            print(f"{csq_dict['Feature']}\t{csq_dict['BIOTYPE']}\t{csq_dict['CANONICAL']}")      
            found = True
            break
    if found:
        break

Transcripts
C	chr1982663	T
ENST00000631376.1	lncRNA	YES


For features we show `SYMBOL` if we have it, `Feature`, `BIOTYPE` if we have it and if there is no `SYMBOL`, and finally `STRAND` providing it is either 1 (`forward strand`) or -1 (`reverse strand`). We need `CANONICAL` for ordering the transcripts. Using 'YES' as our top transcript

In [9]:
print("Features")

reader = open_vcf()

found_g = 0
found_b = 0
found_s = 0
for rec in reader:
    for str_csq in rec.INFO["CSQ"]:
        csq_dict = csq_obj(csq_headers,str_csq.split("|"))
        if csq_dict['Feature'] and csq_dict['Feature_type'] != "Transcript":  
            if csq_dict['SYMBOL'] and found_g == 0:
                print(f"{rec.REF}\t{rec.CHROM}{rec.begin}\t{', '.join([alt.value for alt in rec.ALT])}")
                print(f"Feature:{csq_dict['Feature']} Gene:{csq_dict['SYMBOL']}")
                found_g = 1
            if csq_dict['BIOTYPE'] and found_b == 0:
                print(f"{rec.REF}\t{rec.CHROM}{rec.begin}\t{', '.join([alt.value for alt in rec.ALT])}")
                print(f"Feature:{csq_dict['Feature']} Biotype:{csq_dict['BIOTYPE']}")
                found_b = 1
            if csq_dict['STRAND'] and found_s == 0:
                print(f"{rec.REF}\t{rec.CHROM}{rec.begin}\t{', '.join([alt.value for alt in rec.ALT])}")
                print(f"Feature:{csq_dict['Feature']} Strand: {csq_dict['STRAND']}")
                found_s = 1


    if found_g + found_b + found_s == 3:
        break

Features
A	chr19243500	G
Feature:ENSR00001020616 Biotype:enhancer
C	chr19267212	T
Feature:ENSM00201890131 Strand: 1


We only seem to have gene symbols for transcript feature types!

In [10]:
reader = open_vcf()

counter = 0
for rec in reader:
    if len(rec.ALT) > 1:
        print(f"Alt alleles {len(rec.ALT)} CSQ entries {len(rec.INFO['CSQ'])}-------------------")
        counter += 1
        for str_csq in rec.INFO["CSQ"]:
            csq_dict = csq_obj(csq_headers,str_csq.split("|"))
            print(f"{csq_dict['Allele']}\t{csq_dict['Feature']}\t{csq_dict['Feature_type']}\t{csq_dict['SYMBOL']}")
        
        if counter > 4:
            break

Alt alleles 2 CSQ entries 2-------------------
GTGTGTGTGTGT	ENST00000251287.3	Transcript	HCN2
-	ENST00000251287.3	Transcript	HCN2
Alt alleles 2 CSQ entries 8-------------------
GTGTGT	ENST00000251287.3	Transcript	HCN2
-	ENST00000251287.3	Transcript	HCN2
GTGTGT	ENST00000587057.5	Transcript	POLRMT
-	ENST00000587057.5	Transcript	POLRMT
GTGTGT	ENST00000588649.7	Transcript	POLRMT
-	ENST00000588649.7	Transcript	POLRMT
GTGTGT	ENST00000592633.5	Transcript	POLRMT
-	ENST00000592633.5	Transcript	POLRMT
Alt alleles 2 CSQ entries 10-------------------
ACAC	ENST00000588649.7	Transcript	POLRMT
ACACACACACACAC	ENST00000588649.7	Transcript	POLRMT
ACAC	ENST00000590573.4	Transcript	POLRMT
ACACACACACACAC	ENST00000590573.4	Transcript	POLRMT
ACAC	ENST00000590709.3	Transcript	POLRMT
ACACACACACACAC	ENST00000590709.3	Transcript	POLRMT
ACAC	ENST00000592863.2	Transcript	POLRMT
ACACACACACACAC	ENST00000592863.2	Transcript	POLRMT
ACAC	ENSR00001219077	RegulatoryFeature	
ACACACACACACAC	ENSR00001219077	RegulatoryFeatur

Need to look here to figure out this https://github.com/Ensembl/ensembl-hypsipyle/blob/main/common/schemas/predicted_molecular_consequence.graphql. -> Implemented by variant-allele -> create_allele_predicted_molecular_consequence @ 147. 

**This method only operates on feature_type transcript!**

Looking again I believe the gene and regulation feature column is populated with elements from feature_type `transcript`

For a feature type of `transcript` use `SYMBOL` `Gene` `STRAND` for the feature section.

Something else is shown if `SYMBOL` is missing. I currently can not locate it. I thought it might be `IMPACT` but the only value I could see in the example is `MODIFIER`. However it also seems to be missing from https://github.com/Ensembl/ensembl-web-tools-api/blob/dev/APISpecification.yaml

Nakib from the Variation team shared the following insight 

> That value enhancer , motif are particular to feature types `RegulatoryFeature` , `MotifFeature` and you can find them under `BIOTYPE` field. So, according to current design you have to fill those spot using the `BIOTYPE` for those feature types.

> Although I remember Sarah mentioning we would have gene associated with regulatory/motif features in future and would also have gene symbols in such case. Probably a design change would require (having separate column for regulatory/motif features most likely)


In [11]:
reader = open_vcf()

counter = 0
for rec in reader:
    if len(rec.ALT) > 1:
        print(f"Alt alleles {len(rec.ALT)} CSQ entries {len(rec.INFO['CSQ'])}-------------------")
        counter += 1
        for str_csq in rec.INFO["CSQ"]:
            csq_dict = csq_obj(csq_headers,str_csq.split("|"))
            if csq_dict['Feature_type'] == "Transcript" and csq_dict['Allele'] != "-": 
                print(f"{csq_dict['Allele']}\t{csq_dict['SYMBOL']}\t{csq_dict['Gene']}\t{csq_dict['STRAND']}\t{csq_dict['IMPACT']}")
        
        if counter > 10:
            break

Alt alleles 2 CSQ entries 2-------------------
GTGTGTGTGTGT	HCN2	ENSG00000099822	1	MODIFIER
Alt alleles 2 CSQ entries 8-------------------
GTGTGT	HCN2	ENSG00000099822	1	MODIFIER
GTGTGT	POLRMT	ENSG00000099821	-1	MODIFIER
GTGTGT	POLRMT	ENSG00000099821	-1	MODIFIER
GTGTGT	POLRMT	ENSG00000099821	-1	MODIFIER
Alt alleles 2 CSQ entries 10-------------------
ACAC	POLRMT	ENSG00000099821	-1	MODIFIER
ACACACACACACAC	POLRMT	ENSG00000099821	-1	MODIFIER
ACAC	POLRMT	ENSG00000099821	-1	MODIFIER
ACACACACACACAC	POLRMT	ENSG00000099821	-1	MODIFIER
ACAC	POLRMT	ENSG00000099821	-1	MODIFIER
ACACACACACACAC	POLRMT	ENSG00000099821	-1	MODIFIER
ACAC	POLRMT	ENSG00000099821	-1	MODIFIER
ACACACACACACAC	POLRMT	ENSG00000099821	-1	MODIFIER
Alt alleles 2 CSQ entries 10-------------------
CGCACA	FSTL3	ENSG00000070404	1	MODIFIER
CGCGCGCGCACACACACA	FSTL3	ENSG00000070404	1	MODIFIER
CGCACA	FSTL3	ENSG00000070404	1	MODIFIER
CGCGCGCGCACACACACA	FSTL3	ENSG00000070404	1	MODIFIER
CGCACA	FSTL3	ENSG00000070404	1	MODIFIER
CGCGCGCGCACACACA

Do we get null STRAND for transcript types?

In [12]:
reader = open_vcf()

counter = 0
unique_strand_values = set()
for rec in reader:
    for str_csq in rec.INFO["CSQ"]:
        csq_dict = csq_obj(csq_headers,str_csq.split("|"))
        if csq_dict["Feature_type"] == "Transcript":
            unique_strand_values.add(csq_dict['STRAND'])
    
print(unique_strand_values)

{'-1', '1'}


In [13]:
print("Max alt allele size")

reader = open_vcf()

for rec in reader:
    if len(rec.ALT) > 1:
        longest_alt = len(max([a.value for a in rec.ALT], key=len))
        alts = ", ".join([a.value for a in rec.ALT])
        print(f"For {alts} the longest alt is {longest_alt}")
        break

Max alt allele size
For CGTGTGTGTGTGT, C the longest alt is 13


In [17]:
print("Unique feature types")
reader = open_vcf()

counter = 0
unique_features_values = set()
for rec in reader:

    
    for str_csq in rec.INFO["CSQ"]:
        csq_dict = csq_obj(csq_headers,str_csq.split("|"))
        unique_features_values.add(csq_dict['Feature_type'])
    
print(unique_features_values)


Unique feature types
{'Transcript', '', 'RegulatoryFeature', 'MotifFeature'}


In [24]:
print("intergenic_variant")
reader = open_vcf()
count = 0
for rec in reader:
    for str_csq in rec.INFO["CSQ"]:
        csq_dict = csq_obj(csq_headers,str_csq.split("|"))
        cons = csq_dict['Consequence'].split('&')
        
        if "intergenic_variant" in cons:
            print(f"intergenic_variant Allele:{csq_dict['Allele']}\tGene Symbol:{csq_dict['SYMBOL']}\tGene:{csq_dict['Gene']}\tFeature_type:{csq_dict['Feature_type']}")
            count += 1
    if count > 10: 
        break
    


intergenic_variant
intergenic_variant Allele:A	Gene Symbol:	Gene:	Feature_type:
intergenic_variant Allele:T	Gene Symbol:	Gene:	Feature_type:
intergenic_variant Allele:C	Gene Symbol:	Gene:	Feature_type:
intergenic_variant Allele:G	Gene Symbol:	Gene:	Feature_type:
intergenic_variant Allele:T	Gene Symbol:	Gene:	Feature_type:
intergenic_variant Allele:C	Gene Symbol:	Gene:	Feature_type:
intergenic_variant Allele:A	Gene Symbol:	Gene:	Feature_type:
intergenic_variant Allele:A	Gene Symbol:	Gene:	Feature_type:
intergenic_variant Allele:G	Gene Symbol:	Gene:	Feature_type:
intergenic_variant Allele:C	Gene Symbol:	Gene:	Feature_type:
intergenic_variant Allele:A	Gene Symbol:	Gene:	Feature_type:
